In [1]:
from x3py import x3logging as x3l
from x3py import utils as x3u
from x3py import data as x3d
import glob
import tifffile as tiff
import numpy as np
import SimpleITK as sitk

def readTIFVOL(path):
    FileList = glob.glob(path  + "/*.tiff")
    FileList.sort()
    if not len(FileList):
        x3l.logger.error("No data found!")
        return 0
    
    datavol = []
    
    for e in FileList:
        with tiff.TiffFile(e) as tif:
            datavol.append(tif.pages[0].asarray())
    return np.array(datavol)


def saveTIFVOL(datavol, path, filename):
    for i, image in enumerate(datavol):
        print(image)
        filename = path + filename + f"_{i}.tiff"
        tiff.imwrite(filename, image)
        #x3l.logger.info(filename, "written on disk")

In [2]:
epoints = 20
prefix = "/local/data/alberto/P-1C_"

def getFolderList(prefix,epoints):
    """
    folder_list[0]: Reference dataset.
    folder_list[1:]: Datasets to be registered
    """
    folder_list = glob.glob(prefix + "*")
    folder_list.sort()
    
    #To avoid errors in case of files
    for el in folder_list:
        if os.path.isdir(el):
            pass
        else:
            folder_list.remove(el)
    
    if not len(folder_list):
        x3l.logger.error("No data found!")
        return 0
    else:
        return folder_list[0], folder_list[1:]


ref_path, mov_paths = getFolderList(prefix,epoints)

In [16]:
def register(ar_fxd,ar_mov):

    R = sitk.ImageRegistrationMethod()
    R.SetMetricAsMeanSquares()
    R.SetMetricSamplingStrategy(R.RANDOM)
    R.SetMetricSamplingPercentage(0.01)
    R.SetOptimizerAsRegularStepGradientDescent(4, 0.01, 100)
    R.SetInitialTransform(sitk.TranslationTransform(ar_fxd.GetDimension()))#initial_transform)#
    R.SetInterpolator(sitk.sitkLinear)

    R.AddCommand(sitk.sitkIterationEvent, lambda: self.command_iteration(R))

    outTx = R.Execute(ar_fxd, ar_mov)

    #Put it out in a logger
    x3l.logger.info("-------")
    x3l.logger.info(outTx)
    x3l.logger.info(f"Optimizer stop condition: {R.GetOptimizerStopConditionDescription()}")
    x3l.logger.info(f" Iteration: {R.GetOptimizerIteration()}")
    x3l.logger.info(f" Metric value: {R.GetMetricValue()}")

    resampler = sitk.ResampleImageFilter()
    resampler.SetReferenceImage(ar_fxd)
    resampler.SetInterpolator(sitk.sitkLinear)
    resampler.SetDefaultPixelValue(1.0)
    resampler.SetTransform(outTx)
    #print(self.moving_tmp_nocrop.shape)
    #self.moving_tmp_nocrop = sitk.GetImageFromArray(self.moving_tmp)
    out = resampler.Execute(ar_mov)
    out = np.reshape(out, [200,600,600])
    out = out.astype('float32')
    return out, R



import h5py
def writeh5REG(file_name,buff, energy,*args):
    """
    Write a HDF5 file. Minimal data structure compatible with tomography reconstructions.
    """
    with h5py.File(file_name,'w') as f:
        a = f.create_group('exchange')
        a.create_dataset('data',data=buff,dtype='float32')
        b = f.create_group('processing')
        c = b.create_group('registration')
        b.create_dataset('energy',data=energy,dtype='float32')
        if len(args) != 0:
            R = args[0]
            b.create_dataset('IsRef',data=False)   
            c.create_dataset('OptimizerStopConditionDescription', \
                        data=args[0].GetOptimizerStopConditionDescription())#,dtype='float32')
            c.create_dataset('Iteration',data=R.GetOptimizerIteration())#,dtype='float32') 
        else:
            b.create_dataset('IsRef',data=True)    
        
#writeh5REG("/local/data/alberto/TEST_REG/test_reg0.h5",out, 6.44,R)

import multiprocessing as mp
from contextlib import closing
def distribute_jobs(func,proj):
    """
    Distribute a func over proj on different cores
    """
    args = []
    pool_size = int(mp.cpu_count()/2)
    chunk_size = int((len(proj) - 1) / pool_size + 1)
    pool_size = int(len(proj) / chunk_size + 1)
    for m in range(pool_size):
        ind_start = int(m * chunk_size)
        ind_end = (m + 1) * chunk_size
        if ind_start >= int(len(proj)):
            break
        if ind_end > len(proj):
            ind_end = int(len(proj))
        args += [range(ind_start, ind_end)]
    #mp.set_start_method('fork')
    with closing(mp.Pool(processes=pool_size)) as p:
        out = p.map_async(func, proj)
    out.get()
    p.close()
    p.join()

In [19]:
ref_data = readTIFVOL(ref_path)
print("Reference data:", ref_path)
print(ref_data.shape)
ref_data = ref_data[200:400,250:850,250:850]
ar_fxd = sitk.GetImageFromArray(ref_data)
#Save reference crop
file_name = ref_path + "Reg.h5"
writeh5REG(file_name,ref_data,0)


enpoints = range(0,epoints-1)
print(enpoints)
#distribute job
def run_reg(enpoints):
    mov_data = readTIFVOL(mov_paths[enpoints])
    #Crop the data & convert into sitk dataformat   
    ar_mov = sitk.GetImageFromArray(mov_data[200:400,250:850,250:850])
    buff, R = register(ar_fxd,ar_mov)
    file_name = mov_paths[enpoints] + "Reg.h5"
    writeh5REG(file_name,buff, enpoints,R)
    print(file_name, "saved")

#for i in range(0,epoints-1):

    
distribute_jobs(run_reg,enpoints)


Reference data: /local/data/alberto/P-1C_258_rec
(876, 1024, 1024)
range(0, 19)
/local/data/alberto/P-1C_263_recReg.h5 saved
/local/data/alberto/P-1C_265_recReg.h5 saved
/local/data/alberto/P-1C_259_recReg.h5 saved
/local/data/alberto/P-1C_261_recReg.h5 saved
/local/data/alberto/P-1C_264_recReg.h5 saved
/local/data/alberto/P-1C_262_recReg.h5 saved
/local/data/alberto/P-1C_260_recReg.h5 saved
/local/data/alberto/P-1C_272_recReg.h5 saved
/local/data/alberto/P-1C_269_recReg.h5 saved
/local/data/alberto/P-1C_268_recReg.h5 saved
/local/data/alberto/P-1C_267_recReg.h5 saved
/local/data/alberto/P-1C_266_recReg.h5 saved
/local/data/alberto/P-1C_274_recReg.h5 saved
/local/data/alberto/P-1C_271_recReg.h5 saved
/local/data/alberto/P-1C_270_recReg.h5 saved
/local/data/alberto/P-1C_275_recReg.h5 saved
/local/data/alberto/P-1C_273_recReg.h5 saved
/local/data/alberto/P-1C_277_recReg.h5 saved
/local/data/alberto/P-1C_276_recReg.h5 saved


In [ ]:
print(out.shape)
      

#saveTIFVOL(out,"/local/data/alberto/TEST_REG/","test_0_")

In [20]:
import x3py

In [53]:
class C:
    def __init__(self, **kwargs):
        vars().update(kwargs)
        print(a,b,c)

In [54]:
def test(*params):
    print(params)

In [55]:


#mylist=[23,1.1,'b']
tst = C(a=23,b=1.1,c='b')

NameError: name 'a' is not defined

In [52]:
test(mylist)

([23, 1.1, 'b'],)
